In [32]:
ftraining = r'C:\P\Machine Learning\bohaci\train.csv'
import pandas as pd
training = pd.read_csv(ftraining,delimiter=',',header=0,names = ['Age', 'Employment','SSIP','Education','EducationLength','MaritalStatus','Occupation','Relationship','Race','Gender','CapitalGain','CapitalLoss','EmploymentHours','NativeCountry','IsRich'])
training.shape

X, y = training.iloc[:,:-1], training.iloc[:, -1]

print('X, y shape: {0},{1}'.format(X.shape,y.shape))

X, y shape: (32561, 14),(32561,)


In [33]:
features = X[['Age','SSIP','EducationLength','CapitalGain','CapitalLoss']].fillna(0)

In [34]:
to_tokenize = ['Employment','Education','MaritalStatus','Occupation','Relationship','Race','Gender','NativeCountry']
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences 
from keras.utils import to_categorical

vocab_size = 100  # total number of feature

#num_words is tne number of unique words in the sequence, if there's more top count words are taken
tokenizers = {}
for t in to_tokenize:
    currentcolumn = X[t].fillna('0')
    tokenizers[t] = Tokenizer(currentcolumn.nunique()+1,filters='')
    
    tokenizers[t].fit_on_texts(currentcolumn)
    sequences = tokenizers[t].texts_to_sequences(currentcolumn)
    features[t] = pad_sequences(sequences, 1)

In [148]:
from keras.models import Model
from keras.backend import expand_dims
from keras.layers import Input,Embedding,Reshape,concatenate, Conv1D, Flatten, Dense,LSTM,MaxPooling1D,Dropout
from keras.regularizers import l2, l1, l1_l2

embedding_output_dim = 8

em, inputs = {}, {}
for name in features:
    vocabulary = features[name].max()+1 #for lookups max = nunique()
    inputs[name] = Input(shape=(1,))
    em[name] = Embedding(vocabulary,embedding_output_dim)(inputs[name])

conv,concatem = [], []
i = 0
for a,b in list(itertools.combinations([key for key in em], 2)):
    print(a,b,i)
    concatem.append(concatenate([em[a],em[b]],axis=1))
    conv.append(Conv1D(filters=32, kernel_size=2,  activation='relu')(concatem[i]))
    i += 1

#kernel_regularizer=l1_l2(0.01)
convmerge = concatenate([con for con in conv],axis=1)
convmerge = Conv1D(filters=64, kernel_size=2,  activation='relu')(convmerge)
convmerge = Conv1D(filters=128, kernel_size=2,  activation='relu')(convmerge)
#convmerge = MaxPooling1D(pool_size=2)(convmerge)
#convmerge = Dropout(0.3)(convmerge)
#convmerge = Conv1D(filters=128, kernel_size=2,  activation='relu')(convmerge)
#convmerge = Conv1D(filters=128, kernel_size=2,  activation='relu')(convmerge)
flatten = Flatten()(convmerge)
d = Dense(1, kernel_initializer='normal', activation='sigmoid')(flatten)

model = Model(inputs=[inputs[name] for name in features], outputs=d)
model.summary()

SSIP Gender 0
SSIP Age 1
SSIP Race 2
SSIP CapitalGain 3
SSIP NativeCountry 4
SSIP Occupation 5
SSIP MaritalStatus 6
SSIP Relationship 7
SSIP CapitalLoss 8
SSIP EducationLength 9
SSIP Employment 10
SSIP Education 11
Gender Age 12
Gender Race 13
Gender CapitalGain 14
Gender NativeCountry 15
Gender Occupation 16
Gender MaritalStatus 17
Gender Relationship 18
Gender CapitalLoss 19
Gender EducationLength 20
Gender Employment 21
Gender Education 22
Age Race 23
Age CapitalGain 24
Age NativeCountry 25
Age Occupation 26
Age MaritalStatus 27
Age Relationship 28
Age CapitalLoss 29
Age EducationLength 30
Age Employment 31
Age Education 32
Race CapitalGain 33
Race NativeCountry 34
Race Occupation 35
Race MaritalStatus 36
Race Relationship 37
Race CapitalLoss 38
Race EducationLength 39
Race Employment 40
Race Education 41
CapitalGain NativeCountry 42
CapitalGain Occupation 43
CapitalGain MaritalStatus 44
CapitalGain Relationship 45
CapitalGain CapitalLoss 46
CapitalGain EducationLength 47
CapitalGai

In [149]:
from keras.optimizers import Adamax, SGD
adam = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [150]:
import numpy as np
batch_size = 32
history = model.fit(x = [features[name] for name in features], y = np.array(y), batch_size = batch_size, epochs = 1, validation_split=.1)

Train on 29304 samples, validate on 3257 samples
Epoch 1/1
29304/29304 [==============================] - 317s - loss: 0.3145 - acc: 0.8565 - val_loss: 0.2867 - val_acc: 0.8695


In [135]:
history.history['val_acc']

[0.86613447938113264]

In [112]:
model.save(r'C:\P\Machine Learning\cotoctes\convAll32,64,128Pooliter')

In [77]:
#from keras.models import load_model
#model = load_model(r'C:\P\Machine Learning\cotoctes\convAll32,64,1281iter')

In [86]:
ftraining = r"C:\P\Machine Learning\bohaci\test.csv"
import pandas as pd
testing = pd.read_csv(ftraining,delimiter=',',header=0,names = ['Age', 'Employment','SSIP','Education','EducationLength','MaritalStatus','Occupation','Relationship','Race','Gender','CapitalGain','CapitalLoss','EmploymentHours','NativeCountry'])
featuresT = testing[['Age','SSIP','EducationLength','CapitalGain','CapitalLoss']].fillna(0)

for t in to_tokenize:
    currentcolumn = testing[t].fillna('0')
    
    sequences = tokenizers[t].texts_to_sequences(currentcolumn)
    featuresT[t] = pad_sequences(sequences, 1)
    
#if some of the values is above the current maximum
for n in featuresT:
    origMax, currentMax = 0, 0
    origMax = max(features[n])
    currentMax = max(featuresT[n])
    if(currentMax > origMax):
        featuresT[featuresT[n] > origMax] = origMax
    print('{0}, maxorig: {1}, maxnew {2} / updated {3}'.format(n,origMax,currentMax,max(featuresT[n])))
    
pred = model.predict([featuresT[name] for name in features])

Age, maxorig: 90, maxnew 90 / updated 90
SSIP, maxorig: 1484705, maxnew 1490400 / updated 1484705
EducationLength, maxorig: 16, maxnew 1484705 / updated 16
CapitalGain, maxorig: 99999, maxnew 99999 / updated 99999
CapitalLoss, maxorig: 4356, maxnew 3770 / updated 3770
Employment, maxorig: 9, maxnew 16 / updated 9
Education, maxorig: 16, maxnew 16 / updated 16
MaritalStatus, maxorig: 7, maxnew 9 / updated 7
Occupation, maxorig: 15, maxnew 15 / updated 15
Relationship, maxorig: 6, maxnew 7 / updated 6
Race, maxorig: 5, maxnew 6 / updated 5
Gender, maxorig: 2, maxnew 5 / updated 2
NativeCountry, maxorig: 42, maxnew 41 / updated 41


In [91]:
print(pred.shape)
import csv
with open(r'C:\P\Machine Learning\bohaci\outputconv.csv','w') as f:
    wr = csv.writer(f, delimiter='\n')
    wr.writerow(pred.round().astype(int))

(16281, 1)
